In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [6]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()
'''

"\n# List of categorical columns\nobject_cols = [col for col in features.columns if 'cat' in col]\n\n# ordinal-encode categorical columns\nX = features.copy()\nX_test = test.drop(['id'], axis=1).copy()\nordinal_encoder = OrdinalEncoder()\nX[object_cols] = ordinal_encoder.fit_transform(features[object_cols])\nX_test[object_cols] = ordinal_encoder.transform(test[object_cols])\n\n# Preview the ordinal-encoded features\nX.head()\n"

In [7]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()


,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,ohe_46,ohe_47,ohe_48,ohe_49,ohe_50,ohe_51,ohe_52,ohe_53,ohe_54,ohe_55
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [9]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [10]:
pseudo = pd.read_csv("../input/30-days-pseudo/submission.csv")[target]
train_pseudo = pd.concat([X, y], axis=1)
test_pseudo = pd.concat([X_test, pseudo], axis=1)
all_pseudo = pd.concat([train_pseudo, test_pseudo]).reset_index(drop=True)

# Optuna

In [11]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [12]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo.iloc[:,:-1], y=all_pseudo[target]):
  """
  """
  param_space = {
               #'device':'gpu',  # Use GPU acceleration
               #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda',3.94, 4.44),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 37.7, 38.7),
                'subsample': trial.suggest_uniform('subsample', 0.935, 0.945),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.0415, 0.0515),
                #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
               'learning_rate':trial.suggest_uniform('learning_rate', 1.88e-2, 2.12e-2),
                 'min_child_samples':trial.suggest_int('min_child_samples', 123, 125),
              'num_leaves':trial.suggest_int('num_leaves', 97, 101),
              'max_depth':trial.suggest_int('max_depth', 6, 8),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
               'n_estimators':N_ESTIMATORS,
                'n_jobs' : -1,
              'metric':'rmse',
              'max_bin':trial.suggest_int('max_bin', 890, 902),
              'cat_smooth':trial.suggest_int('cat_smooth', 101, 103),
              'cat_l2':trial.suggest_loguniform('cat_l2', 27.667, 29.667)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo.iloc[trn_idx, :-1], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_row(X_train)

  X_valid, y_valid = all_pseudo.iloc[oof_idx, :-1], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]
  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        #categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
    )


    lgb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, lgb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-30 20:07:15,603] A new study created in memory with name: no-name-4d2d1525-0161-400d-83e8-67b232014486


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.717303
[2000]	valid_0's rmse: 0.711784
[3000]	valid_0's rmse: 0.710156
[4000]	valid_0's rmse: 0.709496
[5000]	valid_0's rmse: 0.709183
[6000]	valid_0's rmse: 0.709015
[7000]	valid_0's rmse: 0.7089
[8000]	valid_0's rmse: 0.708833
[9000]	valid_0's rmse: 0.708801
Early stopping, best iteration is:
[9574]	valid_0's rmse: 0.708774
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.71732
[2000]	valid_0's rmse: 0.711781
[3000]	valid_0's rmse: 0.710212
[4000]	valid_0's rmse: 0.709572
[5000]	valid_0's rmse: 0.70922
[6000]	valid_0's rmse: 0.709071
[7000]	valid_0's rmse: 0.708993
[8000]	valid_0's rmse: 0.708937
[9000]	valid_0's rmse: 0.708896
[10000]	valid_0's rmse: 0.708873
[11000]	valid_0's rmse: 0.708851
[12000]	valid_0's rmse: 0.708838
Early stopping, best iteration is:
[12746]	valid_0's rmse: 0.708826


[I 2021-08-30 20:14:36,539] Trial 0 finished with value: 0.708698980652528 and parameters: {'reg_lambda': 4.253509628445695, 'reg_alpha': 37.77882041973822, 'subsample': 0.9437095661518056, 'colsample_bytree': 0.04580230801735675, 'learning_rate': 0.021087420995892447, 'min_child_samples': 124, 'num_leaves': 101, 'max_depth': 8, 'max_bin': 891, 'cat_smooth': 101, 'cat_l2': 28.40575682043659}. Best is trial 0 with value: 0.708698980652528.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.718113
[2000]	valid_0's rmse: 0.712273
[3000]	valid_0's rmse: 0.710455
[4000]	valid_0's rmse: 0.709681
[5000]	valid_0's rmse: 0.709302
[6000]	valid_0's rmse: 0.70909
[7000]	valid_0's rmse: 0.708964
[8000]	valid_0's rmse: 0.708881
[9000]	valid_0's rmse: 0.70883
Early stopping, best iteration is:
[9552]	valid_0's rmse: 0.708799
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.718038
[2000]	valid_0's rmse: 0.71216
[3000]	valid_0's rmse: 0.710397
[4000]	valid_0's rmse: 0.709655
[5000]	valid_0's rmse: 0.709241
[6000]	valid_0's rmse: 0.70906
[7000]	valid_0's rmse: 0.708957
[8000]	valid_0's rmse: 0.708886
[9000]	valid_0's rmse: 0.708841
[10000]	valid_0's rmse: 0.70881
Early stopping, best iteration is:
[10644]	valid_0's rmse: 0.708805


[I 2021-08-30 20:21:19,981] Trial 1 finished with value: 0.7087062738230423 and parameters: {'reg_lambda': 3.9584371071905484, 'reg_alpha': 38.006884197482954, 'subsample': 0.9355559094496743, 'colsample_bytree': 0.04579018334936816, 'learning_rate': 0.01946722242170515, 'min_child_samples': 123, 'num_leaves': 99, 'max_depth': 8, 'max_bin': 893, 'cat_smooth': 103, 'cat_l2': 28.913195462136645}. Best is trial 0 with value: 0.708698980652528.


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.718748
[2000]	valid_0's rmse: 0.712736
[3000]	valid_0's rmse: 0.710771
[4000]	valid_0's rmse: 0.709914
[5000]	valid_0's rmse: 0.70943
[6000]	valid_0's rmse: 0.709172
[7000]	valid_0's rmse: 0.709005
[8000]	valid_0's rmse: 0.708899
[9000]	valid_0's rmse: 0.708824
[10000]	valid_0's rmse: 0.708756
[11000]	valid_0's rmse: 0.708717
[12000]	valid_0's rmse: 0.708694
[13000]	valid_0's rmse: 0.708675
Early stopping, best iteration is:
[12810]	valid_0's rmse: 0.708675
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.718695
[2000]	valid_0's rmse: 0.712703
[3000]	valid_0's rmse: 0.710785
[4000]	valid_0's rmse: 0.70993
[5000]	valid_0's rmse: 0.709426
[6000]	valid_0's rmse: 0.709175
[7000]	valid_0's rmse: 0.709025
[8000]	valid_0's rmse: 0.708909
[9000]	valid_0's rmse: 0.70884
[10000]	valid_0's rmse: 0.708795
[11000]	valid_0's rmse: 0.708771
[12000]	valid_0's rmse: 0.708745
[130

[I 2021-08-31 00:38:10,731] Trial 28 finished with value: 0.7085957400610862 and parameters: {'reg_lambda': 4.304564832707003, 'reg_alpha': 38.25550936395965, 'subsample': 0.9350963507518101, 'colsample_bytree': 0.04164594433152002, 'learning_rate': 0.019172734795609048, 'min_child_samples': 125, 'num_leaves': 98, 'max_depth': 7, 'max_bin': 896, 'cat_smooth': 103, 'cat_l2': 27.69885913898252}. Best is trial 24 with value: 0.7085541107346746.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.719408
[2000]	valid_0's rmse: 0.713298
[3000]	valid_0's rmse: 0.711246
[4000]	valid_0's rmse: 0.710296
[5000]	valid_0's rmse: 0.709713
[6000]	valid_0's rmse: 0.709383
[7000]	valid_0's rmse: 0.709171
[8000]	valid_0's rmse: 0.709017
[9000]	valid_0's rmse: 0.708927
[10000]	valid_0's rmse: 0.708845
[11000]	valid_0's rmse: 0.708786
[12000]	valid_0's rmse: 0.708731
[13000]	valid_0's rmse: 0.708692
[14000]	valid_0's rmse: 0.708676
[15000]	valid_0's rmse: 0.708648
[16000]	valid_0's rmse: 0.708633
[17000]	valid_0's rmse: 0.708625
[18000]	valid_0's rmse: 0.708612
[19000]	valid_0's rmse: 0.708604
Early stopping, best iteration is:
[19013]	valid_0's rmse: 0.708604
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.719326
[2000]	valid_0's rmse: 0.713294
[3000]	valid_0's rmse: 0.711248
[4000]	valid_0's rmse: 0.710259
[5000]	valid_0's rmse: 0.709656
[6000]	valid_0's rmse: 0.70932

[I 2021-08-31 00:49:17,718] Trial 29 finished with value: 0.7085582844366513 and parameters: {'reg_lambda': 4.262111612446379, 'reg_alpha': 38.4549183900258, 'subsample': 0.936772126782439, 'colsample_bytree': 0.04183013187247322, 'learning_rate': 0.019771557581118045, 'min_child_samples': 125, 'num_leaves': 100, 'max_depth': 6, 'max_bin': 896, 'cat_smooth': 103, 'cat_l2': 27.80159201912716}. Best is trial 24 with value: 0.7085541107346746.


Number of finished trials: 30
Best trial: {'reg_lambda': 4.204365659614207, 'reg_alpha': 38.26193447233029, 'subsample': 0.9362199207253972, 'colsample_bytree': 0.04159141923763232, 'learning_rate': 0.02007338012388031, 'min_child_samples': 125, 'num_leaves': 99, 'max_depth': 6, 'max_bin': 896, 'cat_smooth': 103, 'cat_l2': 27.681741352232027}


In [14]:
study.best_params

{'reg_lambda': 4.204365659614207,
 'reg_alpha': 38.26193447233029,
 'subsample': 0.9362199207253972,
 'colsample_bytree': 0.04159141923763232,
 'learning_rate': 0.02007338012388031,
 'min_child_samples': 125,
 'num_leaves': 99,
 'max_depth': 6,
 'max_bin': 896,
 'cat_smooth': 103,
 'cat_l2': 27.681741352232027}

# Log

====== Ordinal encoding =========

0.709167665584924 no noise ver9

0.7087162719562731 no noise ver16 (narrow space)

0.7086508785373714 no noise ver17 (narrow space) final

0.7127333982112984 row-wise noise ver10

0.7126600249836035 random noise ver11

====== One-hot encoding =========

0.7205030924594267 no noise ver13

0.722057702839371 random noise ver14

0.7222305639305339 row-wise noise ver15

0.7090416357652942 no noise ver27

0.7087695012696115 no noise ver28 (narrow space)

0.708670251857603 no noise ver29 (narrow space)

0.7086084505246009 no noise ver30 (narrow space)

0.7086017221244755 no noise ver31 (narrow space)

0.7085567775516957 no noise ver32 (narrow space)

==========================================
=========== Changed pseudo ===============

====== Ordinal encoding =========

0.709174903761106 no noise ver20

0.7086337003244899 no noise ver21 (narrow space)

0.708623028878119 no noise ver22 (narrow space)

0.7085074339931049 no noise ver24 (narrow space) final